In [36]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from datetime import date

### Extract data

In [14]:
# start and end dates
start_date = datetime(2025, 1, 1)
end_date = datetime.today()

In [16]:
# list for saving results
all_data = []

In [17]:
# cycle by day
current_date = start_date
while current_date <= end_date:
    date_str = current_date.strftime("%Y%m%d")
    url = f"https://bank.gov.ua/NBUStatService/v1/statdirectory/exchange?date={date_str}&json"
    
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        for d in data:
            all_data.append({
                "date": d["exchangedate"],
                "currency_code": d["cc"],
                "currency_name": d["txt"],
                "rate": d["rate"]
            })
    else:
        print(f"Помилка {response.status_code} для {date_str}")
    
    current_date += timedelta(days=1)

In [18]:
# convert to DataFrame
df = pd.DataFrame(all_data)

In [24]:
# save to file
df.to_csv("nbu_exchange_rates.csv", index=False, encoding="utf-8-sig")

### Data verification and cleaning

In [20]:
# number of rows and columns
print(df.shape)

(13678, 4)


In [21]:
# information about data types
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13678 entries, 0 to 13677
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           13678 non-null  object 
 1   currency_code  13678 non-null  object 
 2   currency_name  13678 non-null  object 
 3   rate           13678 non-null  float64
dtypes: float64(1), object(3)
memory usage: 427.6+ KB
None


In [9]:
# unique currency codes
print(df["currency_code"].unique())

['AUD' 'CAD' 'CNY' 'CZK' 'DKK' 'HKD' 'HUF' 'INR' 'IDR' 'ILS' 'JPY' 'KZT'
 'KRW' 'MXN' 'MDL' 'NZD' 'NOK' 'RUB' 'SGD' 'ZAR' 'SEK' 'CHF' 'EGP' 'GBP'
 'USD' 'BYN' 'AZN' 'RON' 'TRY' 'XDR' 'BGN' 'EUR' 'PLN' 'DZD' 'BDT' 'AMD'
 'DOP' 'IRR' 'IQD' 'KGS' 'LBP' 'LYD' 'MYR' 'MAD' 'PKR' 'SAR' 'VND' 'THB'
 'AED' 'TND' 'UZS' 'TWD' 'TMT' 'RSD' 'TJS' 'GEL' 'BRL' 'XAU' 'XAG' 'XPT'
 'XPD']


In [10]:
# check NULL
print(df.isnull().sum())

date             0
currency_code    0
currency_name    0
rate             0
dtype: int64


In [11]:
# duplicate check
duplicates = df[df.duplicated()]
print(duplicates)

Empty DataFrame
Columns: [date, currency_code, currency_name, rate]
Index: []


In [23]:
# Сheck if there are any problematic values
print("Rows with rate <= 0 or NaN:")
print(df[(df["rate"] <= 0) | (df["rate"].isna())])

Rows with rate <= 0 or NaN:
Empty DataFrame
Columns: [date, currency_code, currency_name, rate]
Index: []


## Data from the official NBU API and load it into PostgreSQL

In [44]:
# PostgreSQL connection parameters
conn = psycopg2.connect(
    dbname="my_projects",
    user="postgres",
    password="991Gfhjkm.",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

In [45]:
# Start and end date
start_date = date(2025, 1, 1)
end_date = date.today()

In [46]:
current_date = start_date
while current_date <= end_date:
    url = f"https://bank.gov.ua/NBUStatService/v1/statdirectory/exchange?date={current_date.strftime('%Y%m%d')}&json"
    response = requests.get(url).json()

    for item in response:
        cur.execute(
            """
            INSERT INTO nbu_courses (date, currency_code, currency_name, rate)
            VALUES (%s, %s, %s, %s)
            ON CONFLICT (date, currency_code) DO NOTHING;
            """,
            (current_date, item["cc"], item["txt"], item["rate"])
        )

    current_date += timedelta(days=1)

In [47]:
# Save changes
conn.commit()
cur.close()
conn.close()